In [ ]:
%run imputers/interpolate.ipynb
%run imputers/median.ipynb
%run imputers/mode.ipynb
%run imputers/mean.ipynb
%run imputers/knn.ipynb
%run imputers/hotdeck.ipynb

import asyncio

imputers = {
    "interpolation": {
        "title": "Interpolation",
        "function": interpolate
    },
    "median imputation": {
        "title": "Median imputation",
        "function": median_imputation
    },
    "mode imputation": {
        "title": "Mode imputation",
        "function": mode_imputation
    },
    "mean imputation": {
        "title": "Mean imputation",
        "function": mean_imputation
    },
    "KNN imputation": {
        "title": "K-Nearest-Neighbors imputation",
        "function": KNNImputation
    },
    "hotdeck imputation": {
        "title": "Hot-deck imputation",
        "function": hotdeck
    }
}

imputer_select = widgets.Dropdown(
    options=list(imputers.keys()),
    value=None,
    description='Imputer:'
)
display(imputer_select)
imputer = None
imputed_dfs: [pd.DataFrame] = None

imputation_status = widgets.HTML(value="")
display(imputation_status)


async def do_imputation():
    global imputed_dfs, imputer
    imputed_dfs = []
    for i in range(len(dfs_with_gaps)):
        imputation_status.value = f"Running imputation... ({i}/{len(dfs_with_gaps)})"
        dataset_config['current_gap_indices'] = gaps_indices[i]
        result = imputer["function"](dfs_with_gaps[i], dataset_config)
        if asyncio.iscoroutine(result):
            result = await result
        imputed_dfs.append(result)
    imputation_status.value = "Imputation complete"


def on_imputer_select_change(evt):
    global imputer
    imputer = imputers[evt.new]
    asyncio.ensure_future(do_imputation())

imputer_select.observe(on_imputer_select_change, 'value')
